In [3]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [4]:
import chromadb
import uuid

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name='portfolio')

if not collection.count():
    for _, row in df.iterrows():
       collection.add(documents=row['Techstack'],
                      metadatas={'links': row['Links']},
                      ids=[str(uuid.uuid4())])

In [5]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model_name="llama-3.1-8b-instant",
    groq_api_key='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',
    temperature=0.0,
)

response=llm.invoke("The first person to land on a moon was...")
print(response.content)

The first person to land on the Moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the Moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first person to set foot on the Moon.


In [6]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.disneycareers.com/job/bristol/associate-project-manager-design-and-delivery/391/79656190512")
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.
















Associate Project Manager Design & Delivery at DISNEY



































































































Skip Navigation





Visit the new Disney Careers!
EN





Career Areas

The Walt Disney Company offers an array of opportunities. Select a career area to see all available jobs.


Business Support

Administration
Asset Management
Business Strategy and Development
Government Affairs
Human Resources
Legal and Business Affairs
Project Management
Social Responsibility
View All Business Support Jobs



Consumer Goods

Licensing
Merchandising
Publishing
View All Consumer Goods Jobs



Creative

Animation and Visual Effects
Creative
Gaming and Interactive
Graphic Design
Marketing and Digital Media
View All Creative Jobs



Finance, Data & Analytics

Banking
Data Science and Analytics
Finance and Accounting
View All Finance, Data & Analytics Jobs



Guest / Customer Service

Call Center
Culinary
Food and Beverage
Hotel and Resor

In [7]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ### INSTRUCTION:
    The scraped text is from the career's page of a website.
    Your job is to extract the job postings and return them in JSON format containing following keys:
    'role','experience','skills' and 'description'. Do not include any responsibilites in description, only ABOUT the job.
    Only return the valid JSON. Do not give the name of the job in the beginning, only in the role. Do not put any text except the keys and their corresponding values. 
    ### VALID JSON (NO PREAMBLE):
    """
)

chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data':page_data})
print(res.content)

[
  {
    "role": "Associate Project Manager Design & Delivery",
    "experience": "5 years of experience in project design and/or construction of tenant improvement work or 3 years Disney project coordination experience.",
    "skills": "Proficiency in Microsoft Office Suite and project management software, Familiarity with project planning, scheduling, budgeting, and cost tracking, Strong organizational and problem-solving skills with the ability to manage multiple tasks.",
    "description": "Corporate Real Estate (CRE) is responsible for the strategic support of The Walt Disney Company’s (TWDC) Global Real Estate portfolio, providing subject matter expertise in the areas of design and construction to our key stakeholders."
  }
]


In [8]:
from langchain_core.output_parsers import JsonOutputParser

jParser = JsonOutputParser()
jRes = jParser.parse(res.content)
jRes

[{'role': 'Associate Project Manager Design & Delivery',
  'experience': '5 years of experience in project design and/or construction of tenant improvement work or 3 years Disney project coordination experience.',
  'skills': 'Proficiency in Microsoft Office Suite and project management software, Familiarity with project planning, scheduling, budgeting, and cost tracking, Strong organizational and problem-solving skills with the ability to manage multiple tasks.',
  'description': 'Corporate Real Estate (CRE) is responsible for the strategic support of The Walt Disney Company’s (TWDC) Global Real Estate portfolio, providing subject matter expertise in the areas of design and construction to our key stakeholders.'}]

In [11]:
job = jRes[0]
job['skills']

'Proficiency in Microsoft Office Suite and project management software, Familiarity with project planning, scheduling, budgeting, and cost tracking, Strong organizational and problem-solving skills with the ability to manage multiple tasks.'

In [13]:
links = collection.query(query_texts=[job['skills']], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/xamarin-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}]]

In [17]:
prompt_letter = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION
        {job_description}
        ### INSTRUCTION
        Using the job description provided, write a cover letter that suits the requirements of the given company. Make sure that the cover 
        letter is brief, but provides all the necessary details. Start with something like: Hello, my name is Dias Daurenuly... 
        Also add the most relevant ones from the following links to showcase your portfolio {link_list}
        Do not provide a preamble.
        ### COVER LETTER (NO PREAMBLE):
        
        """
)

chain_letter = prompt_letter | llm
res = chain_letter.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Dear Hiring Manager,

My name is Dias Daurenuly, and I am excited to apply for the Associate Project Manager Design & Delivery position in the Corporate Real Estate department at The Walt Disney Company. With 5 years of experience in project design and construction of tenant improvement work, I am confident that my skills and expertise align with the requirements of this role.

As a seasoned project manager, I possess proficiency in Microsoft Office Suite and project management software, which enables me to efficiently plan, schedule, and track projects. My strong organizational and problem-solving skills allow me to manage multiple tasks and prioritize deadlines. I am also familiar with project planning, budgeting, and cost tracking, which I believe are essential skills for this position.

In addition to my technical skills, I have a proven track record of delivering projects on time and within budget. I am proud to share my portfolio, which showcases my expertise in project managemen